<a href="https://colab.research.google.com/github/Dharshan4038/BMI-Calculator/blob/main/Ontology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %pip install PyMuPDF easyocr pandas tabula-py langchain langchain-openai openai

In [ ]:
# %pip install PyPDF2==2.10.5

In [ ]:
import fitz  # PyMuPDF for PDF handling
import easyocr  # EasyOCR for image text extraction
import camelot  # Camelot for table extraction
import os

In [ ]:
import re
import fitz  # PyMuPDF for PDF handling

def clean_text(text):
    # Remove extra newlines, tabs, and multiple spaces
    text = re.sub(r'\s+', ' ', text)
    # Strip leading and trailing spaces
    text = text.strip()
    return text

In [94]:
reader = easyocr.Reader(['en'])

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""

    # Loop through each page
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)

        # Extract text from page
        full_text += page.get_text()

        # Extract images from the page
        image_list = page.get_images(full=True)

        for img_index, img in enumerate(image_list):
            xref = img[0]  # Get image XREF
            base_image = doc.extract_image(xref)  # Extract the image
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            image_filename = f"/content/drive/MyDrive/Ontology/pdf_images/page_{page_num}_img_{img_index}.{image_ext}"

            # Save the image to disk
            with open(image_filename, "wb") as img_file:
                img_file.write(image_bytes)

            # Perform OCR on the extracted image
            ocr_result = reader.readtext(image_filename, detail=0)
            full_text += "\n".join(ocr_result) + "\n"


    return full_text

pdf_path = "/content/drive/MyDrive/RAG/test_case.pdf"
pdf_text = extract_text_from_pdf(pdf_path)
pdf_text = clean_text(pdf_text)
# print(pdf_text)

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

In [ ]:
def clean_table(df):
    # Clean each column and row by removing unwanted characters
    df = df.applymap(lambda x: re.sub(r'\s+', ' ', str(x)).strip() if isinstance(x, str) else x)
    return df

In [95]:
def extract_tables_from_pdf(pdf_path):
    # Use Camelot to extract tables
    tables = camelot.read_pdf(pdf_path, pages="all", flavor='stream')

    cleaned_tables = []

    # Process and clean each table
    for i, table in enumerate(tables):
        df = table.df  # Extract table as Pandas DataFrame
        cleaned_df = clean_table(df)  # Clean the DataFrame
        cleaned_tables.append(cleaned_df)

        # Optionally save the cleaned table to CSV
        cleaned_df.to_csv(f"/content/drive/MyDrive/Ontology/tables_csv/cleaned_table_{i}.csv", index=False)

    return cleaned_tables

pdf_path = "/content/drive/MyDrive/RAG/test_case.pdf"
cleaned_tables = extract_tables_from_pdf(pdf_path)

# # Display cleaned tables
# for i, cleaned_table in enumerate(cleaned_tables):
#     print(f"Cleaned Table {i}:")
#     print(cleaned_table)

/usr/local/lib/python3.10/dist-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/usr/local/lib/python3.10/dist-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 2
  warnings.warn(f"No tables found in table area {table_idx + 1}")
<ipython-input-28-05ca7958f328>:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: re.sub(r'\s+', ' ', str(x)).strip() if isinstance(x, str) else x)


In [96]:
combined_data = {
    "pdf_text": pdf_text,
    "tables": cleaned_tables
}

# # Print the combined cleaned data
# print("Extracted Text from PDF:\n", combined_data['pdf_text'])
# for i, table in enumerate(combined_data['tables']):
#     # print(f"\nCleaned Table {i}:\n", table)

## Creating Agent to generate Ontology

In [84]:
import openai
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
import os
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""


# Initialize the OpenAI LLM for Azure OpenAI
llm = AzureChatOpenAI(
    azure_endpoint="",
    api_key="",
    model = "gpt-35-turbo",
    max_tokens=1000,
    azure_deployment="GPT-35-TURBO",
    api_version="2024-06-01"
)

In [104]:
# # Define a prompt template for generating the ontology template
# ontology_prompt = PromptTemplate(
#     input_variables=["pdf_text", "tables"],
#     template="""
#       Based on the following extracted content from the PDF, including tables and images,
#       identify key entities (classes), relationships, properties, instances. It should contain
#       only 5 important classes and its properties and relationships, instances.
#       Then, generate an ontology in RDF/Turtle format.

#       PDF text: {pdf_text}
#       Tables: {tables}

#       Please generate only the ontology template without mapping specific data points. I dont want you to give any explanation
#       I want only the ontology template.

#       eg: RDF/Turtle ontology template:
     # # @prefix ex: <http://example.org/ontology#> .
#         # Classes
#         ex:TestingTool a rdfs:Class .
#         ex:Algorithm a rdfs:Class .
#         ex:Dataset a rdfs:Class .

#         # Properties
#         ex:usesDataset a rdf:Property ;
#             rdfs:domain ex:TestingTool ;
#             rdfs:range ex:Dataset .

#         ex:implementsAlgorithm a rdf:Property ;
#             rdfs:domain ex:TestingTool ;
#             rdfs:range ex:Algorithm .

#         # Example Instances
#         ex:EvoSuite a ex:TestingTool ;
#             ex:implementsAlgorithm ex:GeneticAlgorithm ;
#             ex:usesDataset ex:SF100Corpus .

#         ex:GeneticAlgorithm a ex:Algorithm .

#         ex:SF100Corpus a ex:Dataset ;
#             rdfs:label "SF100 Corpus" .

#             """
# )

# # Create an LLM chain using the prompt and LLM
# ontology_chain = ontology_prompt | llm

In [105]:
# # Function to run the agent and generate ontology
# def generate_ontology(cleaned_data):
#     pdf_text = cleaned_data['pdf_text'][:5000] #Truncate to the first 5000 characters
#     tables = "\n".join([str(table) for table in cleaned_data['tables'][:5]])  # Convert tables to string and use only the first 5

#     # Run the RunnableSequence (use invoke, not run)
#     ontology_template = ontology_chain.invoke({
#         "pdf_text": pdf_text,
#         "tables": tables
#     })

#     return ontology_template

In [91]:
# # Assuming you have the cleaned data from the PDF stored in 'combined_data'
# combined_data = {
#     "pdf_text": pdf_text,        # Cleaned text from PDF
#     "tables": cleaned_tables     # Cleaned tables from the PDF
# }

# # Generate the ontology template using Azure OpenAI
# ontology_template = generate_ontology(combined_data)

# # Print the generated ontology template
# print(ontology_template.content)

@prefix ex: <http://example.org/ontology#> .

# Classes
ex:Software a rdfs:Class .
ex:TestingTool a rdfs:Class .
ex:TestSuite a rdfs:Class .
ex:Dataset a rdfs:Class .
ex:ResearchPaper a rdfs:Class .

# Properties
ex:implements a rdf:Property ;
    rdfs:domain ex:Software ;
    rdfs:range ex:TestingTool .

ex:hasTestSuite a rdf:Property ;
    rdfs:domain ex:Software ;
    rdfs:range ex:TestSuite .

ex:usesDataset a rdf:Property ;
    rdfs:domain ex:TestingTool ;
    rdfs:range ex:Dataset .

ex:hasResearchPaper a rdf:Property ;
    rdfs:domain ex:Software ;
    rdfs:range ex:ResearchPaper .

# Example Instances
ex:SoftwareTool1 a ex:Software ;
    ex:implements ex:TestingTool1 ;
    ex:hasTestSuite ex:TestSuite1 ;
    ex:usesDataset ex:Dataset1 ;
    ex:hasResearchPaper ex:ResearchPaper1 .

ex:TestingTool1 a ex:TestingTool .

ex:TestSuite1 a ex:TestSuite .

ex:Dataset1 a ex:Dataset .

ex:ResearchPaper1 a ex:ResearchPaper .


In [93]:
# Save the generated ontology to a .ttl file
with open("/content/drive/MyDrive/Ontology/Ontology_Template/ontology_template.ttl", "w") as f:
    f.write(ontology_template.content)

## Using Groq API

In [100]:
groq_api_key = "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"

In [98]:
%pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.5 MB/s eta 0:00:00


In [147]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="Gemma2-9b-It",
               groq_api_key=groq_api_key,
               temperature=0.3,
               max_tokens=8192,
               top_p=0.2
      )
llm

/usr/local/lib/python3.10/dist-packages/langchain_groq/chat_models.py:355: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7e424cc03340>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7e424d3bbd60>, model_name='Gemma2-9b-It', temperature=0.3, model_kwargs={'top_p': 0.2}, groq_api_key=SecretStr('**********'), max_tokens=8192)

In [148]:
# Define a prompt template for generating the ontology template
ontology_prompt = PromptTemplate(
    input_variables=["pdf_text", "tables"],
    template="""
      Based on the following extracted content from the PDF, including tables and images,
      identify key entities (classes), relationships, properties, instances. It should contain
      only 5 important classes and its properties and relationships, instances.
      Then, generate an ontology in RDF/Turtle format.

      PDF text: {pdf_text}
      Tables: {tables}

      Please generate only the ontology template without mapping specific data points. I dont want you to give any explanation
      I want only the ontology template.

      eg: RDF/Turtle ontology template:
      @prefix ex: <http://example.org/ontology#> .
        # Classes
        ex:TestingTool a rdfs:Class .
        ex:Algorithm a rdfs:Class .
        ex:Dataset a rdfs:Class .

        # Properties
        ex:usesDataset a rdf:Property ;
            rdfs:domain ex:TestingTool ;
            rdfs:range ex:Dataset .

        ex:implementsAlgorithm a rdf:Property ;
            rdfs:domain ex:TestingTool ;
            rdfs:range ex:Algorithm .

        # Example Instances
        ex:EvoSuite a ex:TestingTool ;
            ex:implementsAlgorithm ex:GeneticAlgorithm ;
            ex:usesDataset ex:SF100Corpus .

        ex:GeneticAlgorithm a ex:Algorithm .

        ex:SF100Corpus a ex:Dataset ;
            rdfs:label "SF100 Corpus" .

            """
)

# Create an LLM chain using the prompt and LLM
ontology_chain = ontology_prompt | llm

In [149]:
# Function to run the agent and generate ontology
def generate_ontology(cleaned_data):
    pdf_text = cleaned_data['pdf_text'][:10000] #Truncate to the first 5000 characters
    tables = "\n".join([str(table) for table in cleaned_data['tables'][:5]])  # Convert tables to string and use only the first 5

    # Run the RunnableSequence (use invoke, not run)
    ontology_template = ontology_chain.invoke({
        "pdf_text": pdf_text,
        "tables": tables
    })

    return ontology_template

In [150]:
# Assuming you have the cleaned data from the PDF stored in 'combined_data'
combined_data = {
    "pdf_text": pdf_text,        # Cleaned text from PDF
    "tables": cleaned_tables     # Cleaned tables from the PDF
}

# Generate the ontology template using Azure OpenAI
ontology_template = generate_ontology(combined_data)

# Print the generated ontology template
print(ontology_template.content)

```turtle
@prefix ex: <http://example.org/ontology#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .

# Classes
ex:TestTool a rdfs:Class .
ex:TestTechnique a rdfs:Class .
ex:Dataset a rdfs:Class .
ex:Algorithm a rdfs:Class .
ex:ResearchPaper a rdfs:Class .

# Properties
ex:usesTechnique a rdf:Property ;
    rdfs:domain ex:TestTool ;
    rdfs:range ex:TestTechnique .

ex:basedOn a rdf:Property ;
    rdfs:domain ex:ResearchPaper ;
    rdfs:range ex:TestTool .

ex:employsAlgorithm a rdf:Property ;
    rdfs:domain ex:TestTool ;
    rdfs:range ex:Algorithm .

ex:usesDataset a rdf:Property ;
    rdfs:domain ex:TestTool ;
    rdfs:range ex:Dataset .

ex:publishedIn a rdf:Property ;
    rdfs:domain ex:ResearchPaper ;
    rdfs:range ex:DigitalLibrary .

ex:hasTitle a rdf:Property ;
    rdfs:domain ex:ResearchPaper ;
    rdfs:range xsd:string .

# Example Instances
ex:EvoSuite a ex:TestTool ;
    ex:usesTechnique ex:GeneticAl

## Mixtral Model

In [173]:
from langchain_groq import ChatGroq
model = ChatGroq(model="mixtral-8x7b-32768",
               groq_api_key=groq_api_key,
               temperature=0.3,
               max_tokens=32768,
               top_p=0.2
      )
model

/usr/local/lib/python3.10/dist-packages/langchain_groq/chat_models.py:355: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7e424cd70d30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7e424cbb45e0>, temperature=0.3, model_kwargs={'top_p': 0.2}, groq_api_key=SecretStr('**********'), max_tokens=32768)

In [182]:
# Define a prompt template for generating the ontology template
ontology_prompt = PromptTemplate(
    input_variables=["pdf_text", "tables"],
    template="""
      Based on the following extracted content from the PDF, including tables and images,
      identify key entities (classes), relationships, properties, instances. It should contain
      all the classes and its properties and relationships, instances.
      Then, generate an ontology in RDF/Turtle format.

      PDF text: {pdf_text}
      Tables: {tables}

      Please generate only the ontology template without mapping specific data points. I dont want you to give any explanation
      I want only the ontology template.

      eg: RDF/Turtle ontology template:
      @prefix ex: <http://example.org/ontology#> .
        # Classes
        ex:TestingTool a rdfs:Class .
        ex:Algorithm a rdfs:Class .
        ex:Dataset a rdfs:Class .

        # Properties
        ex:usesDataset a rdf:Property ;
            rdfs:domain ex:TestingTool ;
            rdfs:range ex:Dataset .

        ex:implementsAlgorithm a rdf:Property ;
            rdfs:domain ex:TestingTool ;
            rdfs:range ex:Algorithm .

        # Example Instances
        ex:EvoSuite a ex:TestingTool ;
            ex:implementsAlgorithm ex:GeneticAlgorithm ;
            ex:usesDataset ex:SF100Corpus .

        ex:GeneticAlgorithm a ex:Algorithm .

        ex:SF100Corpus a ex:Dataset ;
            rdfs:label "SF100 Corpus" .

            """
)

# Create an LLM chain using the prompt and LLM
ontology_chain = ontology_prompt | model

In [195]:
import time

def generate_ontology_mixtral(cleaned_data, rate_limit_tpm):
    pdf_text = cleaned_data['pdf_text'][:5000]
    tables = "\n".join([str(table) for table in cleaned_data['tables'][:5]])

    # Calculate time to wait between requests
    wait_time = 60 / rate_limit_tpm

    # Generate ontology template
    ontology_template = ontology_chain.invoke({
        "pdf_text": pdf_text,
        "tables": tables
    })

    # Sleep to respect rate limit
    time.sleep(wait_time)

    return ontology_template

In [196]:
# Assuming you have the cleaned data from the PDF stored in 'combined_data'
combined_data = {
    "pdf_text": pdf_text,        # Cleaned text from PDF
    "tables": cleaned_tables     # Cleaned tables from the PDF
}

# Generate the ontology template using Azure OpenAI
ontology_template = generate_ontology_mixtral(combined_data,rate_limit_tpm=5000)

# Print the generated ontology template
print(ontology_template.content)

Here is the RDF/Turtle ontology template based on the provided content:

@prefix ex: <http://example.org/ontology#> .

# Classes
ex:TestingTool a rdfs:Class .
ex:Category a rdfs:Class .
ex:Journal a rdfs:Class .
ex:Conference a rdfs:Class .
ex:Approach a rdfs:Class .
ex:Activity a rdfs:Class .
ex:Dataset a rdfs:Class .
ex:Software a rdfs:Class .

# Properties
ex:assignedToCategory a rdf:Property ;
rdfs:domain ex:TestingTool ;
rdfs:range ex:Category .

ex:publishedIn a rdf:Property ;
rdfs:domain ex:ResearchPaper ;
rdfs:range ex:Publication .

ex:presentedAt a rdf:Property ;
rdfs:domain ex:ResearchPaper ;
rdfs:range ex:Conference .

ex:hasApproach a rdf:Property ;
rdfs:domain ex:ResearchPaper ;
rdfs:range ex:Approach .

ex:hasActivity a rdf:Property ;
rdfs:domain ex:Approach ;
rdfs:range ex:Activity .

ex:usesDataset a rdf:Property ;
rdfs:domain ex:Approach ;
rdfs:range ex:Dataset .

ex:implementedIn a rdf:Property ;
rdfs:domain ex:Software ;
rdfs:range ex:TestingTool .

# Example Instan